# 特異スペクトル変換法による変化点検知

In [0]:
import numpy as np
import matplotlib.pyplot as plt

In [0]:
# Keoghらの心電図のデータ
!wget http://www.cs.ucr.edu/~eamonn/discords/qtdbsel102.txt

In [0]:
data = np.loadtxt("./qtdbsel102.txt",delimiter="\t")

train_data = data[1:3000, 2]
test_data = data[3001:6000, 2]

In [0]:
w = 50 # width
m = 2
k = int(w/2)
L = int(k/2) # lag
Tt = test_data.size
score = np.zeros(Tt)

In [0]:
def embed(lst, dim):
    emb = np.empty((0,dim), float)
    for i in range(lst.size - dim + 1):
        tmp = np.array(lst[i:i+dim]).reshape((1,-1))
        emb = np.append( emb, tmp, axis=0)
    return emb

In [0]:
for t in range(w+k, Tt-L+1+1):
    tstart = t-w-k+1
    tend = t-1
    X1 = embed(test_data[tstart:tend], w).T[::-1, :] # 履歴行列
    X2 = embed(test_data[(tstart+L):(tend+L)], w).T[::-1, :] # テスト行列

    U1, s1, V1 = np.linalg.svd(X1, full_matrices=True)
    U1 = U1[:,0:m]
    U2, s2, V2 = np.linalg.svd(X2, full_matrices=True)
    U2 = U2[:,0:m]

    U, s, V = np.linalg.svd(U1.T.dot(U2), full_matrices=True)
    sig1 = s[0]
    score[t] = 1 - np.square(sig1)

In [0]:
# 変化度をmax1にするデータ整形
mx = np.max(score)
score = score / mx

# プロット
test_for_plot = data[3001:6000, 2]
fig = plt.figure()
ax1 = fig.add_subplot(111)
ax2 = ax1.twinx()

p1, = ax1.plot(score, '-b')
ax1.set_ylabel('degree of change')
ax1.set_ylim(0, 1.2)
p2, = ax2.plot(test_for_plot, '-g')
ax2.set_ylabel('original')
ax2.set_ylim(0, 12.0)
plt.title("Singular Spectrum Transformation")
ax1.legend([p1, p2], ["degree of change", "original"])
plt.show()